In [ ]:
import numpy as np

nx = 100

x = np.linspace(0, 101, nx)

rho = np.sin(np.pi * x)  # Random charge density

varphi = np.zeros(nx)  # Initial potential array

epsilon = 8.854e-12 * 40.0 # Permittivity of free space (F/m) 

dx = 1

i_a = 1e-6  # Current density (A/m)

sigma = 1.0 # Conductivity (S/m)

# function to calculate the potential, varphi 
def gauss_seidel_1D(varphi, rho, maxiter = 100000, tol = 1e-4):
    
    # Get the length of potential array (1-D)
    nx = len(varphi)
    
    # Iterate until convergence or maximum iterations reached
    for it in range(maxiter):

        varphi_old = varphi.copy()

        varphi[0] = 0.0
        
        varphi[-1] = varphi[-2] + dx * i_a/sigma

        # Update the potential using the Gauss-Seidel formula
        for j in range(1, nx-1):
                varphi[j] = 0.5 * (varphi[j-1] + varphi[j+1] - rho[j] * dx**2 / epsilon)
        
        # Check for convergence
        if np.max(np.abs(varphi - varphi_old)) < tol:
            print(f"Converged after {it} iterations.")
            return varphi
    
    print("Maximum iterations reached without convergence.")
    return varphi

test = gauss_seidel_1D(varphi, rho)

print(test)

In [ ]:
import numpy as np
from scipy.linalg import solve

nx = 5

domain = 5.0

x = np.linspace(0, domain, nx)

rho = np.copy(x) # Initial charge density

varphi = np.zeros(nx)  # Initial potential array

epsilon = 2 #8.854e-12 * 40.0 # Permittivity of free space (F/m) 

dx = 1

i_a = 0  # Current density (A/m)

sigma = 1.0 # Conductivity (S/m)

def gauss(A, b, x, n):
    L = np.tril(A)
    U = A - L
    for i in range(n):
        x = np.dot(np.linalg.inv(L), b - np.dot(U, x))
    return x

A = np.zeros((nx, nx))
b = np.zeros(nx)

#Dirichlet boundary conditions for the electrolyte
A[0, 0] = 1.0
b[0] = 0.0

#Setting up the matrix A and vector b
for i in range(1, nx-1):
    A[i, i-1] = -1.0
    A[i, i] = 2.0
    A[i, i+1] = -1.0
    b[i] = rho[i] * dx**2 / epsilon

#Neumann boundary conditions for the anode
A[-1, -2] = -1.0
A[-1, -1] = 1.0
b[-1] = dx * i_a / sigma

print(A)
print(b)

print (gauss(A, b, x, nx))
print( solve(A, b))

[[ 1.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  1.]]
[0.    0.625 1.25  1.875 0.   ]
[0.         2.77832031 5.21728516 6.74926758 6.74926758]
[0.    3.75  6.875 8.75  8.75 ]


In [ ]:
def gauss_seidel(A, b, x, n, tol=1e-10):
    for it in range(n):
        x_old = x.copy()
        for i in range(len(b)):
            sum1 = np.dot(A[i, :i], x[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x[i] = (b[i] - sum1 - sum2) / A[i, i]
        if np.linalg.norm(x - x_old) < tol:
            print(f"Converged after {it} iterations.")
            return x
    print("Maximum iterations reached without convergence.")
    return x